In [4]:
# Setup: (run this once in your environment)
# !pip install -r requirements.txt

import os, sys, pprint
# Ensure project path is importable
proj_root = os.path.abspath('/mnt/data/ppo_acc_attack')
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

print('Project root:', proj_root)
print('Files:')
print('\n'.join(sorted(os.listdir(proj_root))))


Project root: /mnt/data/ppo_acc_attack
Files:


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/ppo_acc_attack'

In [ ]:
# Imports and helper wrappers
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from acc_env import ACCEnv
from attacks import FGSMAttack, OIAttack
import os, csv, json

# Simple plotting helper (one plot per cell as required)
def plot_traj(traj, title, out_png=None):
    t = np.arange(len(traj['Δx']))
    plt.figure()
    plt.plot(t, traj['Δx'])
    plt.xlabel('t (steps)'); plt.ylabel('Δx (m)'); plt.title(title + ' — headway (Δx)')
    if out_png: plt.savefig(out_png.replace('.png','_dx.png'), bbox_inches='tight')
    plt.show()

    plt.figure()
    plt.plot(t, traj['v'])
    plt.xlabel('t (steps)'); plt.ylabel('v (m/s)'); plt.title(title + ' — ego speed (v)')
    if out_png: plt.savefig(out_png.replace('.png','_v.png'), bbox_inches='tight')
    plt.show()

    plt.figure()
    plt.plot(t, traj['a'])
    plt.xlabel('t (steps)'); plt.ylabel('a (m/s^2)'); plt.title(title + ' — acceleration (a)')
    if out_png: plt.savefig(out_png.replace('.png','_a.png'), bbox_inches='tight')
    plt.show()

print('Imports OK')


In [ ]:
# Environment factory (vectorized-friendly)

def make_env(brake_profile=False, normalize_obs=True, seed=0):
    def _thunk():
        return ACCEnv(brake_profile=brake_profile, normalize_obs=normalize_obs, seed=seed)
    return _thunk

# Quick smoke test
env = make_env(brake_profile=True, normalize_obs=True)()
obs, info = env.reset()
print('Initial observation:', obs)
print('Observation space:', env.observation_space)
print('Action space:', env.action_space)


In [ ]:
# Training cell - adjust TOTAL_STEPS if you want a fast demo
from stable_baselines3.common.logger import configure

TOTAL_STEPS = 200_000  # paper suggests >200k; change to 10_000 for quick tests
LOGDIR = 'runs/ppo_demo'

os.makedirs(LOGDIR, exist_ok=True)

# Create VecNormalize-wrapped env (single-threaded for simplicity)
env = DummyVecEnv([make_env(brake_profile=False, normalize_obs=True)])
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=1.0)

model = PPO('MlpPolicy', env, verbose=1, seed=0,
            n_steps=1024, batch_size=128, learning_rate=3e-4, gamma=0.99,
            gae_lambda=0.95, clip_range=0.2, ent_coef=0.0)
new_logger = configure(LOGDIR, ['stdout','csv','tensorboard'])
model.set_logger(new_logger)

print('Starting training for', TOTAL_STEPS, 'steps...')
# Uncomment the following lines to actually run training
# model.learn(total_timesteps=TOTAL_STEPS)
# model.save(os.path.join(LOGDIR,'ppo_acc'))
# env.save(os.path.join(LOGDIR,'vecnormalize.pkl'))

print('Training cell prepared (learn call is commented out to avoid long runs).')


In [ ]:
# Evaluation utilities
import numpy as np

def load_model_and_env(logdir: str):
    env = DummyVecEnv([make_env(brake_profile=True, normalize_obs=True)])
    env = VecNormalize.load(os.path.join(logdir, 'vecnormalize.pkl'), env)
    env.training = False
    env.norm_reward = False
    model = PPO.load(os.path.join(logdir, 'ppo_acc'))
    return model, env

def run_episode(model, env, attack=None, eps=0.01, render_traj=False):
    obs = env.reset()[0]
    traj = {k: [] for k in ['Δx','v','a','r']}
    total_r = 0.0
    collisions = 0
    rmse_accum = 0.0
    rmse_count = 0

    # Wrap model with attack if requested
    atk = None
    if attack == 'fgsm':
        atk = FGSMAttack(model, epsilon=eps, device='cpu')
    elif attack == 'oia':
        atk = OIAttack(model, epsilon=eps, device='cpu')

    while True:
        if atk is None:
            action, _ = model.predict(obs, deterministic=True)
            obs_in = obs
        else:
            action, obs_in = atk.act(obs)

        obs, reward, term, trunc, info = env.step(action)
        total_r += reward[0] if isinstance(reward, np.ndarray) else reward
        traj['Δx'].append(info[0]['Δx'] if isinstance(info, list) else info['Δx'])
        traj['v'].append(info[0]['v'] if isinstance(info, list) else info['v'])
        traj['a'].append(info[0]['a'] if isinstance(info, list) else info['a'])
        traj['r'].append(reward[0] if isinstance(reward, np.ndarray) else reward)

        if atk is not None:
            diff = (obs_in - obs)
            rmse_accum += float((diff**2).mean())
            rmse_count += 1

        done = bool(term) or bool(trunc)
        if term:
            collisions = 1
        if done:
            break

    jerk = np.mean(np.abs(np.diff(traj['a']))) if len(traj['a']) > 1 else 0.0
    rmse = np.sqrt(rmse_accum / max(1, rmse_count))
    return {'return': total_r, 'collision': collisions, 'jerk': jerk, 'rmse': rmse, 'traj': traj}

def eval_many(model, env, which: str | None, episodes:int=20, eps=0.01, out_prefix='artifacts'):
    atk = 'none' if which is None else which
    rets, cols, jerks, rmses = [], [], [], []
    sample_traj = None
    for ep in range(episodes):
        res = run_episode(model, env, attack=which, eps=eps)
        rets.append(res['return'])
        cols.append(res['collision'])
        jerks.append(res['jerk'])
        rmses.append(res['rmse'])
        if sample_traj is None:
            sample_traj = res['traj']
    avg = {
        'avg_return': float(np.mean(rets)),
        'collision_rate': float(np.mean(cols)),
        'avg_jerk': float(np.mean(jerks)),
        'avg_rmse': float(np.mean(rmses)),
    }
    os.makedirs(out_prefix, exist_ok=True)
    out_csv = os.path.join(out_prefix, f"metrics_{atk}.csv")
    with open(out_csv, 'w', newline='') as f:
        w = csv.writer(f)
        w.writerow(['metric','value'])
        for k,v in avg.items():
            w.writerow([k,v])
    if sample_traj is not None:
        plot_traj(sample_traj, f'{atk.upper()} sample episode', os.path.join(out_prefix, f'{atk}_traj.png'))
    print(f"{atk}: {avg}")
    return avg

print('Evaluation functions prepared')
